In [ ]:
import numpy as np
gt_2d = np.load('data_2d_h36m_gt.npz', allow_pickle=True)['positions_2d'].item()

In [ ]:
gt_2d['S9']['Directions'][0][0]

In [ ]:
np.mean(gt_2d['S1']['Directions'][0][0], axis = 0)

In [ ]:
!pip install pillow

In [ ]:
min = np.min(gt_2d['S1']['Directions 1'][0][600], axis=0)
max = np.max(gt_2d['S1']['Directions 1'][0][600], axis=0)

In [ ]:
centroid = (max + min)/2

In [ ]:
radius = np.linalg.norm(max - min)/10

In [ ]:
from PIL import Image
import PIL
import shutil
import os
from os import path
import matplotlib.pyplot as plt

img_path = '/human36m/Videos_S1/S1/images/Directions 1.54138969/0600.jpg'
img = plt.imread(img_path)
fig, ax = plt.subplots()
ax.imshow(img)
plt.axis('off')
circle = plt.Circle((centroid[0], centroid[1]), radius, color='black')
ax.imshow(img)
ax.add_patch(circle)
plt.axis('off')

In [ ]:
!pip install moviepy

In [ ]:
import os
import moviepy.video.io.ImageSequenceClip
from PIL import Image
import PIL
import shutil

def get_occlusion(gt_2d, radius_level, time_level, c):
  # actions = ['Directions 1', 'Directions 2', 'Discussion 2', 'Discussion 3', 'Eating', 'Eating 1', 'Greeting 1', 'Greeting 2', 'Phoning', 'Phoning 1']
  actions = ['Greeting 1', 'Greeting 2', 'Phoning', 'Phoning 1']
  # cams = {'54138969': [0, 1000, 1002], '55011271': [1, 1000, 1000], '58860488': [2, 1000, 1000], '60457274': [3, 1000, 1002]}
  cams = {'55011271': [1, 1000, 1000], '58860488': [2, 1000, 1000], '60457274': [3, 1000, 1002]}

  if radius_level == 1:
      ratio = 10
  elif radius_level == 2:
      ratio = 5
  elif radius_level == 3:
      ratio = 10/3

  for action in actions:
    for cam in cams.keys():
      vid_length = len(gt_2d['S9'][action][cams[cam][0]])
      frame_count = int(vid_length*time_level/5)
      bounding_min = np.min(gt_2d['S9'][action][cams[cam][0]][int(vid_length/2 - frame_count/2)], axis=0)
      bounding_max = np.max(gt_2d['S9'][action][cams[cam][0]][int(vid_length/2 - frame_count/2)], axis=0)
      centroid = (bounding_max + bounding_min)/2
      radius = np.linalg.norm(bounding_max - bounding_min)/ratio
      src_dir = '/human36m/Videos_S9/S9/images/' + action + '.' + cam + '/'
      output_path = '/human36m/Videos_S9/S9/synthetic_occlusion_images/' + action + '.' + cam + '_' + str(radius_level) + str(time_level) + '/'
      if path.exists(output_path) == False:
        files=os.listdir(src_dir)
        shutil.copytree(src_dir, output_path)
    
      print(action)
      print(cam)
      print(radius_level)
      print(time_level)
      print('start: ' + str(int(vid_length/2 - frame_count/2)))
      print('end: ' + str(int(vid_length/2 + frame_count/2)))
      print('radius: ' + str (radius))
      print('centroid: ' + str(centroid))
        
      for img_index in range(int(vid_length/2 - frame_count/2), int(vid_length/2 + frame_count/2)):
        img_path = src_dir + str(img_index).zfill(4) + '.jpg'
        img = plt.imread(img_path)
        fig = plt.figure(frameon=False)
        fig.set_size_inches(cams[cam][1]/100, cams[cam][2]/100)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(img,aspect='auto')
        circle = plt.Circle((centroid[0], centroid[1]), radius, color=c)
        ax.add_patch(circle)

        # print(img_index)
        plt.savefig(output_path + str(img_index).zfill(4) + '.jpg', dpi=100)
        
        plt.close()

In [ ]:
import os
import moviepy.video.io.ImageSequenceClip

for radius_level in [1]:
  for time_level in [1]:
    get_occlusion(gt_2d, radius_level, time_level, 'black' )